In [1]:
from tsplearn import *
import numpy as np 
import pandas as pd

prob_T=0.8

# Load the graph
G = EnhancedGraph(n=40, p_edges=0.162, p_triangles=prob_T, seed=0)
B1 = G.get_b1()
B2 = G.get_b2()

# Sub-sampling if needed to decrease complexity
sub_size = 100
B1 = B1[:, :sub_size]
B2 = B2[:sub_size, :]
B2 = B2[:,np.sum(np.abs(B2), 0) == 3]
nu = B2.shape[1]
nd = B1.shape[1]
T = int(np.ceil(nu*(1-prob_T)))

# Laplacians
Lu, Ld, L = G.get_laplacians(sub_size=100)
n =  L.shape[0]


# Problem and Dictionary Dimensionalities
dictionary_type="separated"
m_train = 150 # Number of Train Signals
m_test = 80 # Number of Test Signal
s = 3 # Number of Kernels (Sub-dictionaries)
k = 2 # Polynomial order
sparsity = .1 # Sparsity percentage
K0_max = 20 #floor(n*sparsity) # Sparsity
sparsity_mode = "max"
n_search = 3000

# Data-Independent Problem Hyperparameters
dictionary_type = ""
K0_coll = np.arange(5, 26, 4) 
max_iter = 30 
patience = 5 
tol = 1e-7 # tolerance for Patience
n_sim = 10
lambda_ = 1e-7 # l2 multiplier
verbose = True

In [2]:
T

13

In [3]:
D_true, Y_train, Y_test, X_train, X_test, epsilon_true, c_true = generate_data(dictionary_type,
                                                                                Lu,
                                                                                Ld,
                                                                                n,
                                                                                s,
                                                                                k,
                                                                                n_sim,
                                                                                m_test,
                                                                                m_train,
                                                                                K0_max,
                                                                                n_search,
                                                                                sparsity_mode,
                                                                                )

  0%|          | 0/3000 [00:00<?, ?it/s]


UnboundLocalError: local variable 'D' referenced before assignment

In [40]:
import dill
import os

path = os.getcwd()
dill.load_session(path+'\\results\\joint\\ipynb_env.db')

In [42]:
import scipy.linalg as sla
import numpy as np
import numpy.linalg as la
import cvxpy as cp
from tsplearn.data_gen import *
from typing import Tuple

def topological_dictionary_learn(Y_train: np.ndarray,
                                 Y_test: np.ndarray, 
                                 K: int, 
                                 n: int, 
                                 s: int,
                                 D0: np.ndarray, 
                                 X0: np.ndarray, 
                                 Lu: np.ndarray, 
                                 Ld: np.ndarray,
                                 dictionary_type: str, 
                                 c: float, 
                                 epsilon: float, 
                                 K0: int,
                                 lambda_: float = 1e-3, 
                                 max_iter: int = 10, 
                                 patience: int = 10,
                                 tol: float = 1e-7, 
                                 verbose: int = 0) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Dictionary learning algorithm implementation for sparse representations of a signal on complex regular cellular.
    The algorithm consists of an iterative alternating optimization procedure defined in two steps: the positive semi-definite programming step
    for obtaining the coefficients and dictionary based on Hodge theory, and the Orthogonal Matching Pursuit step for constructing 
    the K0-sparse solution from the dictionary found in the previous step, which best approximates the original signal.
    Args:
        Y_train (np.ndarray): Training data.
        Y_test (np.ndarray): Testing data.
        K (int): Max order of the polynomial for the single sub-dictionary.
        n (int): Number of data points (number of nodes in the data graph).
        s (int): Number of kernels (sub-dictionaries).
        D0 (np.ndarray): Initial dictionary.
        X0 (np.ndarray): Initial sparse representation.
        Lu (np.ndarray): Upper Laplacian matrix
        Ld (np.ndarray): Lower Laplacian matrix
        dictionary_type (str): Type of dictionary.
        c (float): Boundary constant from the synthetic data generation process.
        epsilon (float): Boundary constant from the synthetic data generation process.
        K0 (int): Sparsity of the signal representation.
        lambda_ (float, optional): Regularization parameter. Defaults to 1e-3.
        max_iter (int, optional): Maximum number of iterations. Defaults to 10.
        patience (int, optional): Patience for early stopping. Defaults to 10.
        tol (float, optional): Tolerance value. Defaults to 1e-7.
        verbose (int, optional): Verbosity level. Defaults to 0.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
         minimum training error, minimum testing error, optimal coefficients, optimal testing sparse representation, and optimal training sparse representation.
    """

    # Define hyperparameters
    min_error_train_norm, min_error_test_norm = 1e20, 1e20
    m_test, m_train = Y_test.shape[1], Y_train.shape[1]
    iter_, pat_iter = 1, 0

    if dictionary_type != "fourier":
        if dictionary_type=="joint":
            Lk, _, _ = compute_Lk_and_lambdak(Lu + Ld, K)
        elif dictionary_type=="edge_laplacian":
            Lk, _, _ = compute_Lk_and_lambdak(Ld, K)
        elif dictionary_type=="separated":
            Luk, _, _ = compute_Lk_and_lambdak(Lu, K, separated=True)
            Ldk, _, _ = compute_Lk_and_lambdak(Ld, K, separated=True)

        # Init the dictionary and the sparse representation 
        D_coll = [cp.Constant(D0[:,(n*i):(n*(i+1))]) for i in range(s)]
        Y = cp.Constant(Y_train)
        X_train = X0
        
        while pat_iter < patience and iter_ <= max_iter:
            
            # SDP Step
            # Init constants and parameters
            D_coll = [cp.Constant(np.zeros((n, n))) for i in range(s)] 
            Dsum = cp.Constant(np.zeros((n, n)))
            X = cp.Constant(X_train)
            I = cp.Constant(np.eye(n))
            
            # Define the objective function
            if dictionary_type in ["joint", "edge_laplacian"]:
                # Init the variables
                h = cp.Variable((s, K))
                hI = cp.Variable((s, 1))
                for i in range(0,s):
                    tmp =  cp.Constant(np.zeros((n, n)))
                    for j in range(0,K):
                        tmp += (cp.Constant(Lk[j, :, :]) * h[i,j])
                    tmp += (I*hI[i])
                    D_coll[i] = tmp
                    Dsum += tmp
                D = cp.hstack([D_coll[i]for i in range(s)])
                term1 = cp.square(cp.norm((Y - D @ X), 'fro'))
                term2 = cp.square(cp.norm(h, 'fro')*lambda_)
                term3 = cp.square(cp.norm(hI, 'fro')*lambda_)
                obj = cp.Minimize(term1 + term2 + term3)

            else:
                # Init the variables
                hI = cp.Variable((s, K))
                hS = cp.Variable((s, K))
                hH = cp.Variable((s, 1))
                for i in range(0,s):
                    tmp =  cp.Constant(np.zeros((n, n)))
                    for j in range(0,K):
                        tmp += ((cp.Constant(Luk[j, :, :])*hS[i,j]) + (cp.Constant(Ldk[j, :, :])*hI[i,j]))
                    tmp += (I*hH[i])
                    D_coll[i] = tmp
                    Dsum += tmp
                D = cp.hstack([D_coll[i]for i in range(s)])
                
                term1 = cp.square(cp.norm((Y - D @ X), 'fro'))
                term2 = cp.square(cp.norm(hI, 'fro')*lambda_)
                term3 = cp.square(cp.norm(hS, 'fro')*lambda_)
                term4 = cp.square(cp.norm(hH, 'fro')*lambda_)
                obj = cp.Minimize(term1 + term2 + term3 + term4)

            # Define the constraints
            constraints = [D_coll[i] >> 0 for i in range(s)] + \
                            [(cp.multiply(c, I) - D_coll[i]) >> 0 for i in range(s)] + \
                            [(Dsum - cp.multiply((c - epsilon), I)) >> 0, (cp.multiply((c + epsilon), I) - Dsum) >> 0]

            prob = cp.Problem(obj, constraints)
            prob.solve(solver=cp.MOSEK, verbose=False)
            # Update the dictionary
            D = D.value

            # OMP Step
            dd = la.norm(D, axis=0)
            W = np.diag(1. / dd)
            Domp = D @ W
            X_train = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_train)
            X_test = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_test)
            # Normalization
            X_train = W @ X_train
            X_test = W @ X_test

            # Error Updating
            error_train_norm = (1/m_train)* np.sum(la.norm(Y_train - (D @ X_train), axis=0)**2 /
                                    la.norm(Y_train, axis=0)**2)
            error_test_norm = (1/m_test)* np.sum(la.norm(Y_test - (D @ X_test), axis=0)**2 /
                                    la.norm(Y_test, axis=0)**2)

            
            # Error Storing
            if (error_train_norm < min_error_train_norm) and (abs(error_train_norm) > np.finfo(float).eps) and (abs(error_train_norm - min_error_train_norm) > tol):
                X_opt_train = X_train
                min_error_train_norm = error_train_norm

            if (error_test_norm < min_error_test_norm) and (abs(error_test_norm) > np.finfo(float).eps) and (abs(error_test_norm - min_error_test_norm) > tol):
                h_opt = h.value if dictionary_type in ["joint", "edge_laplacian"] else [hS.value, hI.value, hH.value]
                D_opt = D
                X_opt_test = X_test
                min_error_test_norm = error_test_norm
                pat_iter = 0
                if verbose == 1:
                    print("New Best Test Error:", min_error_test_norm)
            else:
                pat_iter += 1

            # print("-"*100)
            # print(f'Iter: {iter_}')
            # print()
            # print(f'hH.shape: {hH.shape}')
            # print(f'hH: {hH.value}')
            # print()
            # print(f'hS.shape: {hS.shape}')
            # print(f'hS: {hS.value}')
            # print()
            # print(f'hI.shape: {hI.shape}')
            # print(f'hI: {hI.value}')
            # print()
            # print(f'test error: {error_test_norm}')
            # print()
            # print("-"*100)

            iter_ += 1
    
    else:
        # Fourier Dictionary Benchmark
        L = Lu + Ld
        _, D_opt = sla.eigh(L)
        dd = la.norm(D_opt, axis=0)
        W = np.diag(1./dd)  
        D_opt = D_opt / la.norm(D_opt)
        Domp = D_opt@W
        X_opt_train = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp.real, col=x), axis=0, arr=Y_train)
        X_opt_test = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp.real, col=x), axis=0, arr=Y_test)
        X_opt_train = W @ X_opt_train
        X_opt_test = W @ X_opt_test
        # Error Updating
        min_error_train_norm = (1/m_train)* np.sum(la.norm(Y_train - (D_opt @ X_opt_train), axis=0)**2 /
                                la.norm(Y_train, axis=0)**2)
        min_error_test_norm = (1/m_test)* np.sum(la.norm(Y_test - (D_opt @ X_opt_test), axis=0)**2 /
                                la.norm(Y_test, axis=0)**2)
        h_opt = 0
        
    return min_error_train_norm, min_error_test_norm, h_opt, X_opt_test, X_opt_train, D_opt

In [37]:
c_true

array([1.49493901, 9.97980896, 4.36082596, 5.68226973, 9.7411766 ,
       2.96568358, 9.01249083, 8.3587013 , 9.22004083, 4.87029145])

In [43]:
c = c_true[7]  
epsilon = epsilon_true[7] 
k0 = K0_coll[1]

D0, X0, _ = initialize_dic(Lu, Ld, s, k, Y_train[:, :, 7], k0, dictionary_type, c, epsilon, "only_X")

min_error_train_norm, min_error_test_norm, h_opt, X_opt_test, X_opt_train, D_opt = topological_dictionary_learn(Y_train[:,:,7], Y_test[:,:,7],
                                                                                                                        k, n, s, D0, X0, Lu, Ld, "separated",
                                                                                                                        c, epsilon, k0, lambda_, max_iter,
                                                                                                                        patience, tol)

C:\Users\engri\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_omp.py:420: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(


In [44]:
h_opt

[array([[ 0.06606493,  0.01967646],
        [-0.14494447, -0.03232593],
        [ 0.07207933,  0.01575976]]),
 array([[ 0.07860698,  0.02082813],
        [-0.21199992, -0.00436066],
        [ 0.12997382,  0.00831749]]),
 array([[1.41647626e-12],
        [6.17015881e+00],
        [1.85799240e-11]])]

In [25]:
sigmas = pd.DataFrame({"b2": [B2]})
sigmas

,b2
0,"[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [55]:
def sparse_transform(D, K0, Y_test, Y_train=None):

    # OMP Step
    dd = la.norm(D, axis=0)
    W = np.diag(1. / dd)
    Domp = D @ W
    X_test = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_test)
    # Normalization
    X_test = W @ X_test

    # Same for the training set
    if Y_train!=None:
        X_train = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_train)
        X_train = W @ X_train

        return X_test, X_train
    
    return X_test

In [68]:
def nmse(D, X, Y, m):
    return (1/m)* np.sum(la.norm(Y - (D @ X), axis=0)**2 /la.norm(Y, axis=0)**2)

In [64]:
# global B2, h_opt, k

Ldk,_,_= compute_Lk_and_lambdak(Ld, k)


def indicator_matrix(row):
    row.sigma[row.idx] = 0
    return np.diag(row.sigma)

def compute_Luk(row, b2, k):
    Lu = b2 @ row.sigma @ b2.T
    Luk = np.array([la.matrix_power(Lu, i) for i in range(1, k + 1)])
    return Luk

T = B2.shape[1]
sigmas = pd.DataFrame({"idx": np.arange(T)})

sigmas["sigma"] = sigmas.idx.apply(lambda x: np.ones(T))
sigmas["sigma"] = sigmas.apply(lambda x: indicator_matrix(x), axis=1)
sigmas["Luk"] = sigmas.apply(lambda x: compute_Luk(x, B2, k), axis=1)
sigmas["D"] = sigmas.apply(lambda x: generate_dictionary(h_opt, 1, x.Luk, Ldk), axis=1)
sigmas["X"] = sigmas.D.apply(lambda x: sparse_transform(x, k0, Y_test[:,:,7]))
sigmas["NMSE"] = sigmas.apply(lambda x: nmse(x.D, x.X, Y_test[:,:,7], m_test), axis=1)

In [69]:
sigmas["NMSE"] = sigmas.apply(lambda x: nmse(x.D, x.X, Y_test[:,:,7], m_test), axis=1)

In [72]:
min_error_test_norm

0.0072992054231631855

In [71]:
sigmas.NMSE.min()

0.008666916305215688

In [66]:
sigmas.X[T-3].shape

(100, 80)